In [121]:
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)



In [123]:
import json
import random
import uuid

train = {
    "version":'2.0',
    "data":[{"title":"Custom", "paragraphs":[]}]
}
dev = {
    "version":'2.0',
    "data":[{"title":"Custom", "paragraphs":[]}]
}

with open('./out/question_answer_spans.csv', 'r') as infile:
    j = 0
    for line in infile.read().splitlines():
        j += 1
        
        # each line is a row representing Q,A,SPANS where Q is a question, A is an answer, 
        # and SPANS are the specific subspans of Q that are answered by the corresponding subspans in A
        split = list(map(lambda x: x.replace("{COMMA}", ","), line.split(",")))
        
        # SPANS are represented in a single string as Q1|Q2|A1|A2, where (Q1,Q2) is the subspan in Q that is answered by (A1,A2), a subspan in A
        # if (A1,A2) is ('',''), this means nothing in A answers (Q1,Q2) in Q
        # split based on |
        spans = split[2].split("|")
        # group sequentially into pairs
        spans = list(pairwise(spans))
        # question spans are the even-indexed pairs
        question_spans = spans[0::4]
        # answer spans are the odd-indexed pairs
        answer_spans = spans[2::4]
        
        for i in range(len(question_spans)):
            question_span = question_spans[i]
            answer_span = answer_spans[i]
            qa = {
                "qas":[{"question":split[0][int(question_span[0]):int(question_span[1])],#split[0],
                        "question_component_offset":0,#int(question_span[0]),
                        "question_component_text":split[0][int(question_span[0]):int(question_span[1])],
                        "id":uuid.uuid4().hex,
                        "is_impossible": answer_span[0] == '' and answer_span[1] == '',
                        "answers":[{"text":split[1][int(answer_span[0]):int(answer_span[1])],
                                   "answer_start":int(answer_span[0])} if answer_span[0] != '' else None]
                       }],
                "context": split[1],
            }
            if len(split[0][int(question_span[0]):int(question_span[1])]) == 0:
                print(split[0])
            if random.random() > 0.3:
                train["data"][0]["paragraphs"].append(qa)
            else:
                dev["data"][0]["paragraphs"].append(qa)
print(j)

5389


In [119]:
with open("./out/qa_squad_train.json","w") as outfile:
    json.dump(train, outfile)
with open("./out/qa_squad_dev.json","w") as outfile:
    json.dump(dev, outfile)
    

In [21]:
len(data["data"][0]["paragraphs"])

688

In [120]:
"||||".split("|")

['', '', '', '', '']